In [120]:
import numpy as np
import math
import sys
import tensorflow as tf

In [121]:
class sigmoid():
    @staticmethod
    def activation(x):
        y = 1 / (1 + np.exp(-x))
        return y
    @staticmethod
    def prime(y):
        f = sigmoid.activation(y)
        return (f * (1 - f))

In [122]:
class softmax:
    @staticmethod
    def activation(x):
        # e_x = np.exp(x).reshape(-1,1)
        e_x = np.exp(x - np.max(x,axis=-1).reshape(-1,1))
        return e_x / e_x.sum(axis=-1).reshape(-1,1)
    @staticmethod
    def primeP(y): #marche pas
        n = y.shape[1]
        print(n,y.shape)
        i = np.identity(n)
        un = np.ones((n,y.shape[1]))
        SM = y.reshape((-1,1))
        print(SM.shape)
        jac = np.diagflat(y) - np.dot(SM, SM.T)
        j = np.dot((i* jac),un)
        # x = np.dot(jac,y.T)
        return j.T
    @staticmethod
    def prime(y):
        return(y * (1-y))

In [123]:
def Softmax(x):
    '''
    Performs the softmax activation on a given set of inputs
    Input: x (N,k) ndarray (N: no. of samples, k: no. of nodes)
    Returns: 
    Note: Works for 2D arrays only(rows for samples, columns for nodes/outputs)
    '''
    max_x = np.amax(x, 1).reshape(x.shape[0],1) # Get the row-wise maximum
    e_x = np.exp(x - max_x ) # For stability
    return e_x / e_x.sum(axis=1, keepdims=True) 

In [124]:
def Softmax_grad(x): # Best implementation (VERY FAST)
    '''Returns the jacobian of the Softmax function for the given set of inputs.
    Inputs:
    x: should be a 2d array where the rows correspond to the samples
        and the columns correspond to the nodes.
    Returns: jacobian
    '''
    s = Softmax(x)
    a = np.eye(s.shape[-1])
    temp1 = np.zeros((s.shape[0], s.shape[1], s.shape[1]),dtype=np.float32)
    temp2 = np.zeros((s.shape[0], s.shape[1], s.shape[1]),dtype=np.float32)
    temp1 = np.einsum('ij,jk->ijk',s,a)
    temp2 = np.einsum('ij,ik->ijk',s,s)
    return temp1-temp2

In [142]:
class BinaryCrossEntropy:
    def __init__(self, p, y,_m,**kwargs):
        self.p = p
        self.y = y
        self.m = _m
        self.sw = kwargs.get('sample_weight',1)

    def normalized(self,a,y):
        self.p = a
        self.y = y
        self.c = self.y.shape[-1] #nb de class
    def metrics(self):
        # r = (-1 / self.m) * (np.sum((self.y * np.log(self.p + 1e-8)) + ((1 - self.y) * (np.log(1 - self.p + 1e-8)))))/self.c
        r = (1 / self.m) * np.sum(self.forward() * self.sw)
        return r
    def forward(self):
    # (-y.log(p) - (1-y).log(1-p))
        f = ((-self.y * np.log(self.p + 1e-8)) - ((1 - self.y) * (np.log(1 - self.p + 1e-8))))/self.c
        return f
    def backward(self):
    # -y/p + (1-y)/(1-p)
        # n = self.p.shape[0]
        return (-self.y/(self.p+1e-8) + (1 - self.y)/(1 - self.p+1e-8)) * self.sw

In [148]:
np.random.seed(120)
y = np.array([[0,0,0,0,1,0,0,0,0,0]])
z = np.random.randn(1,10)/np.sqrt(2/10)
a = softmax.activation(z)
# print(a)
print(z,a,a.sum(axis=-1))

[[ 0.32970771  4.52768052 -0.95453353  5.16596603  1.40572632 -0.1239037
  -2.95130235  0.90576774 -0.41480254 -0.22734566]] [[4.99355964e-03 3.32327797e-01 1.38251979e-03 6.29173675e-01
  1.46460052e-02 3.17255608e-03 1.87709251e-04 8.88361913e-03
  2.37177533e-03 2.86078379e-03]] [1.]


In [149]:
a2 = Softmax(z)
print(a2,a2.sum(axis=-1))

[[4.99355964e-03 3.32327797e-01 1.38251979e-03 6.29173675e-01
  1.46460052e-02 3.17255608e-03 1.87709251e-04 8.88361913e-03
  2.37177533e-03 2.86078379e-03]] [1.]


In [150]:
L = BinaryCrossEntropy(a,y,1)
L.normalized(a,y)
l = L.metrics()
print(l)

0.5643483920494577


In [151]:
dL_da = L.backward()
da_dz = softmax.prime(a)
diff = dL_da*da_dz
print(diff,y.shape)
print(a-y)

[[ 4.99355959e-03  3.32327792e-01  1.38251978e-03  6.29173658e-01
  -9.85353322e-01  3.17255605e-03  1.87709249e-04  8.88361904e-03
   2.37177531e-03  2.86078376e-03]] (1, 10)
[[ 4.99355964e-03  3.32327797e-01  1.38251979e-03  6.29173675e-01
  -9.85353995e-01  3.17255608e-03  1.87709251e-04  8.88361913e-03
   2.37177533e-03  2.86078379e-03]]


In [154]:
da_dz_2 = Softmax_grad(a).reshape(10,10)
# print(da_dz_2,dL_da)
print(np.dot(da_dz_2,dL_da.T).T)

[[ 0.52222413  0.785208    0.52001996  1.25547617 -5.68948762  0.52111116
   0.51929316  0.52461149  0.52062266  0.5209209 ]]


In [155]:
da_dz_3 = softmax.primeP(a)
# print(da_dz_3)
print(np.dot(dL_da,da_dz_3))

10 (1, 10)
(10, 1)
[[-2.83507465e-01 -1.26607178e+01 -7.87769002e-02 -1.33128018e+01
  -8.23454927e-01 -1.80450321e-01 -1.07085955e-02 -5.02392272e-01
  -1.35011462e-01 -1.62768065e-01]]


In [157]:
t_tf = tf.nn.softmax(z)
print(t_tf)

tf.Tensor(
[[4.99355964e-03 3.32327797e-01 1.38251979e-03 6.29173675e-01
  1.46460052e-02 3.17255608e-03 1.87709251e-04 8.88361913e-03
  2.37177533e-03 2.86078379e-03]], shape=(1, 10), dtype=float64)


In [136]:
l_tf = tf.keras.losses.BinaryCrossentropy(from_logits=False)
l_tf(y,t_tf).numpy()

0.5643476765841015

In [137]:
y_true = np.array([[0, 1], [0, 0]])
y_pred = np.array([[-18.6, 0.51], [2.94, -12.8]])
sw = np.array([[0.8],[0.2]])
ypS = sigmoid.activation(y_pred)
# yp = np.clip(y_pred,1e-8,1e8)
bce = BinaryCrossEntropy(ypS,y_true,2,sample_weight = sw)
bce.normalized(ypS,y_true)
bce_f = (bce.forward())
print(np.sum(bce_f)/2)
l_bce = bce.metrics()
print(ypS,l_bce)
t_bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
print(t_bce(y_true,y_pred,sample_weight=[0.8, 0.2]).numpy())

0.8654578909902412
[[8.35839003e-09 6.24806474e-01]
 [9.49788727e-01 2.76076495e-06]] 0.24363857325779462
0.2436385925472071
